In [1]:
import pandas as pd
import numpy as np

In [62]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
init_notebook_mode(connected=True)

### Data load

In [4]:
airport = pd.read_csv("Airports2.csv")

In [8]:
airport.columns

Index(['Origin_airport', 'Destination_airport', 'Origin_city',
       'Destination_city', 'Passengers', 'Seats', 'Flights', 'Distance',
       'Fly_date', 'Origin_population', 'Destination_population',
       'Org_airport_lat', 'Org_airport_long', 'Dest_airport_lat',
       'Dest_airport_long'],
      dtype='object')

In [16]:
data.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long,Origin_state,Destination_state
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN,KS,IA
636,SEA,RDM,"Seattle, WA","Bend, OR",2023,4144,112,228,2000-01-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR
637,SEA,RDM,"Seattle, WA","Bend, OR",2995,4699,127,228,2000-12-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR
638,SEA,RDM,"Seattle, WA","Bend, OR",3134,5328,144,228,2000-09-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR
639,SEA,RDM,"Seattle, WA","Bend, OR",3420,5328,144,228,2000-06-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR


### Feature engineering
We are only interested in data from 2000 till 2009.

In [93]:
data = airport[airport["Fly_date"] >= '2000-01-01']

In [11]:
len(data)

2122293

In [22]:
len(data[['Origin_state', 'Passengers']].drop_duplicates())

759

In [94]:
data['Origin_state'] = data['Origin_city'].apply(lambda x: x.split(',')[1].strip(' '))
data['Destination_state'] = data['Destination_city'].apply(lambda x: x.split(',')[1].strip(' '))
data['Year'] = data['Fly_date'].apply(lambda x: x.split('-')[0].strip(' '))
data['Month'] = data['Fly_date'].apply(lambda x: x.split('-')[1].strip(' '))

/Users/leesa/anaconda/envs/fv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/leesa/anaconda/envs/fv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/leesa/anaconda/envs/fv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

Let's have a look at the average daily passengers from each state in 2009.

In [78]:
df = data[data["Fly_date"] >= '2009-01-01']
df = df[['Origin_state', 'Passengers', 'Fly_date']]
df = df.groupby(['Origin_state', 'Fly_date'])['Passengers'].sum().reset_index()
df.head()

,Origin_state,Fly_date,Passengers
0,AK,2009-01-01,154555
1,AK,2009-02-01,141812
2,AK,2009-03-01,170902
3,AK,2009-04-01,154018
4,AK,2009-05-01,196890


In [84]:
df = df.groupby(['Origin_state'])['Passengers'].mean().reset_index()
df.head()

,Origin_state,Passengers
0,AK,2.084442e+05
1,AL,2.115337e+05
2,AR,1.436639e+05
3,AZ,1.532344e+06
4,CA,4.685130e+06


In [85]:
for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['Origin_state'] + '<br>' +\
    'Passengers '+ df['Passengers']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['Origin_state'],
        z = df['Passengers'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Avg passengers")
        ) ]

layout = dict(
        title = '2009 Average passengers by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict(data=data, layout=layout)

iplot(fig)

It seems there are a lot of passengers from California and Texas travelling to other states.  
Now lets have a look at the change in average air passengers across the years for these two states.

In [90]:
df = data[['Origin_state', 'Passengers', 'Fly_date', 'Year']]
df = df.groupby(['Origin_state', 'Fly_date', 'Year'])['Passengers'].sum().reset_index()
df = df.groupby(['Origin_state', 'Year'])['Passengers'].mean().reset_index()
df.head()

,Origin_state,Year,Passengers
0,AK,2000,197716.333333
1,AK,2001,197246.083333
2,AK,2002,208416.416667
3,AK,2003,205828.083333
4,AK,2004,221796.333333


In [97]:
data1 = go.Scatter(
        x=df[df['Origin_state'] == 'CA']['Year'], 
        y=df[df['Origin_state'] == 'CA']['Passengers']
    )

data2 = go.Scatter(
        x=df[df['Origin_state'] == 'TX']['Year'], 
        y=df[df['Origin_state'] == 'TX']['Passengers']
    )

data = [data1, data2]
fig = go.Figure(data = data)
iplot(fig)

For both the states the change is almost similar. The number decreases till 2002 and then increases until 2007 after which it again starts decreasing. This similarity increased our curiosity to check how the overall national average looks like.

In [ ]:
df3 = data[['Passengers', 'Fly_date', 'Year']]
df3 = df3.groupby(['Fly_date', 'Year'])['Passengers'].sum().reset_index()
df3 = df3.groupby(['Year'])['Passengers'].mean().reset_index()
df3.head()

In [98]:
data = [go.Scatter(
        x=df3['Year'], 
        y=df3['Passengers']
    )]

fig = go.Figure(data = data)
iplot(fig)

To our surprise, the national average follows almost similar trend. This makes us think that there might be some driving factor for this change.